In [32]:
import pandas as pd
import re
import numpy as np
import time
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
import multiprocessing as mp

In [47]:
charter_path = '../../nowdata/charters_2015.pkl'
df_charter1 = pd.read_pickle(charter_path)
df_charter1['WEBTEXT']=df_charter1['WEBTEXT'].fillna('') # turn nan to empty iterable for future convenience

stemmer = PorterStemmer()
stem = stemmer.stem # stemmer function

In [45]:
df_charter1

,NCESSCH,URL,LAT1516,LON1516,AM,AS,BL,HI,HP,TR,...,NUMPAGES,NUMWORDS,DISC_RATIO,INQ_RATIO,ESS_RATIO,PROG_RATIO,INQ_STR,WEBTEXT,CMO_WEBTEXT,UNSCRAPED
0,1.001970e+10,http://www.maef.net/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,41,116647,0.000180,0.003438,0.000557,0.000532,-2.463729,"[(http://www.maef.net/, False, 0, Evening Acad...",,False
1,2.000010e+10,https://education.alaska.gov/DOE_Rolodex/Schoo...,60.796131,-161.765194,167.0,0.0,0.0,0.0,0.0,0.0,...,0,0,NaN,NaN,NaN,NaN,NaN,,,True
2,2.001500e+10,https://www.kgbsd.org/ketchikancharter,55.347001,-131.641191,74.0,37.0,2.0,5.0,4.0,5.0,...,1,7811,0.000384,0.002048,0.000128,0.000384,-2.688587,"[(https://www.kgbsd.org/ketchikancharter, Fals...",,False
3,2.001500e+10,http://www.tongassschool.org/,55.347001,-131.641191,57.0,12.0,4.0,6.0,1.0,11.0,...,117,182011,0.000121,0.004582,0.000099,0.000253,-2.338932,[(http://tongassschool.org/classrooms/3rd-and-...,,False
4,2.001800e+10,https://aquarian.asdk12.org/,61.192407,-149.916872,10.0,11.0,6.0,19.0,2.0,51.0,...,1,3953,0.000000,0.002024,0.000000,0.000000,-2.693837,"[(https://www.asdk12.org/aquarian, False, 0, S...",,False
5,2.001800e+10,https://education.alaska.gov/DOE_Rolodex/Schoo...,61.198100,-149.876000,30.0,16.0,18.0,43.0,18.0,69.0,...,0,0,NaN,NaN,NaN,NaN,NaN,,,True
6,2.001800e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,,,True
7,2.001800e+10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,NaN,NaN,NaN,NaN,NaN,,,True
8,2.001800e+10,http://www.winterberrycharterschool.com/,61.194450,-149.791641,15.0,8.0,5.0,14.0,1.0,26.0,...,33,97522,0.000236,0.004296,0.000236,0.000882,-2.366889,"[(http://winterberrycharterschool.com/, False,...",,False
9,2.001800e+10,http://www.asdk12.org/aboutschools/eagleacademy/,61.319213,-149.579442,2.0,3.0,8.0,6.0,0.0,18.0,...,1,0,NaN,NaN,NaN,NaN,NaN,[(https://www.asdk12.org/aboutschools/eagleaca...,,False


In [41]:
dict_path = '/home/jovyan/work/Charter-school-identities/dicts/'
file_ext = '.txt'
dict_names = ['inquiry', 'discipline', 'ess_valid', 'prog_valid', 'rit_valid'] # enter list of names of txt files holding dict

In [111]:
# loads dictionaries into list
# completes dictionary to include entries with dashes taken out
def load_dict():
    dict_list = []
    for name in dict_names:
        with open(dict_path+name+file_ext) as f: 
            new_dict = f.read().splitlines()
            new_words = []
            for entry in new_dict:
                new_words.append(re.sub('[ /-_]', '', entry))
            new_dict.extend(new_words)
            new_dict = set(new_dict)
            dict_list.append(list(new_dict))
    return dict_list

In [52]:
class Page:
    def __init__(self,p):
        self.url = p[0]
        self.boo = p[1]
        self.depth = p[2]
        self.text = p[3]
    def __repr__(self):
        return self.text
    def __eq__(self, other):
        if isinstance(other, Page):
            return self.text == other.text
        else:
            return False
    def __ne__(self, other):
        return (not self.__eq__(other))
    def __hash__(self):
        return hash(self.__repr__())
    
def dict_precalc(dict_list, mode):
    precalc_list = []
    for keywords in dict_list:  
        large_words = []
        large_lengths = []
        large_first_words = []
        key_words = []
        for entry in keywords:
            if mode:
                word = [stem(x) for x in re.split('\W+|_', entry.lower())]
            else:
                word = re.split('\W+|_', entry.lower())
            key_words.append(word) # listified version of each keyword
            if len(word) >= 3:
                large_words.append(word) # contains only large entries(>2 word)
                large_lengths.append(len(word))
                large_first_words.append(word[0]) # first words of each large entry in dict
        precalc_list.append([large_words, large_lengths, large_first_words, key_words])
    return precalc_list

def dict_count2(key_words, large_words, large_lengths, large_first_words, pages):

    """Returns the hit count with given dictionary on page set.

    pages: set of preprocessed page lists corresponding to an entry of the 'webtext' column
    """
    counts = 0 # number of matches between text_list and custom_dict
    res_length = 0
    # Do dictionary analysis for word chunks of lengths max_entry_length down to 1
    for splitted_phrase in pages:
        for length in range(1, 3):
            if len(splitted_phrase) < length:
                continue # If text chunk is shorter than length of dict entries being matched, there are no matches.
            for i in range(len(splitted_phrase) - length + 1):
                entry = splitted_phrase[i:i+length]
                if entry in key_words:
                    counts += 1
                    res_length += length - 1
        indices = np.transpose(np.nonzero([np.array(splitted_phrase) == first_word for first_word in large_first_words]))
        for ind in indices:
            if ind[1] <= (len(splitted_phrase) - large_lengths[ind[0]]) and large_words[ind[0]] == splitted_phrase[ind[1] : ind[1] + large_lengths[ind[0]]]:
                counts += 1
                res_length += large_lengths[ind[0]] - 1
    return counts, res_length

In [104]:
[[] for _ in range(len(dict_names))]

[[], [], [], [], []]

In [95]:
def create_cols(df_charter, mode = 0):
    
    """Creates a count column for each dictionary file with [FILE_NAME]_COUNT as column name. 
    
    mode: 1 for stemming before matching, 0 for no stemming
    Sets strengths with zero hit count to -6
    """
    dict_list = load_dict()
    counts = [[] for _ in range(len(dict_list))]
    res_list = [[] for _ in range(len(dict_list))]
    num_words = [] # WARNING: hitcount/numwords will not give accurate hit ratio in case of multiple word entries. Calculated strength variables do account for this however.
    precalc_list = dict_precalc(dict_list, mode)
    start = time.time()
    for i, row in enumerate(df_charter['WEBTEXT'].values):
        pages = set([Page(p) for p in row])
        if mode:
            pages = [[stem(x) for x in re.split('\W+|_', p.text)] for p in pages] # preprocess pages in same way as dictionaries should have been in above precalc function
        else:
            pages = [re.split('\W+|_', p.text) for p in pages]
        num_words.append(sum([len(p) for p in pages]))
        for j, d in enumerate(dict_list):
            c, res = dict_count2(precalc_list[j][3], precalc_list[j][0], precalc_list[j][1], precalc_list[j][2], pages)
            counts[j].append(c)
            res_list[j].append(res)
        if i%1000 == 0:
            end = time.time()
            print('Time Elapsed:{:f}, Percent Complete:{:f}'.format(end - start,i*100/len(df_charter)))
            
    df_charter['NUMWORDS'] = np.array(num_words)
    for i, name in enumerate(dict_names):
        df_charter['{}_COUNT'.format(name.upper())] = np.array(counts[i])
        df_charter['{}_RATIO'.format(name.upper())] = np.array(counts[i])/(np.array(num_words) - np.array(res_list[i]))
        df_charter['{}_STR'.format(name.upper())] = np.log10(np.array(counts[i])/(np.array(num_words) - np.array(res_list[i])))
    df_charter.replace([np.inf, -np.inf], -6, inplace = True)
    return df_charter

In [ ]:
df_new = create_cols(df_charter1, 0) # run with no stem

In [ ]:
# run in parallel
with mp.Pool(processes = mp.cpu_count() - 1) as pool:
    results = pool.map(create_cols, [df_charter1[300*i:i*300+300] for i in range(round(len(df_charter1)/300)+1)])

Time Elapsed:0.001138, Percent Complete:0.000000
Time Elapsed:0.001205, Percent Complete:0.000000
Time Elapsed:2.621935, Percent Complete:0.000000
Time Elapsed:0.001107, Percent Complete:0.000000
Time Elapsed:1.616501, Percent Complete:0.000000
Time Elapsed:0.674160, Percent Complete:0.000000
Time Elapsed:0.001350, Percent Complete:0.000000
Time Elapsed:0.001293, Percent Complete:0.000000
Time Elapsed:1.967439, Percent Complete:0.000000
Time Elapsed:1.185897, Percent Complete:0.000000
Time Elapsed:0.001148, Percent Complete:0.000000
Time Elapsed:0.001126, Percent Complete:0.000000
Time Elapsed:0.765583, Percent Complete:0.000000
Time Elapsed:0.001253, Percent Complete:0.000000
Time Elapsed:0.001224, Percent Complete:0.000000
Time Elapsed:1.550370, Percent Complete:0.000000
Time Elapsed:0.001581, Percent Complete:0.000000
Time Elapsed:2.331350, Percent Complete:0.000000
Time Elapsed:0.001387, Percent Complete:0.000000
Time Elapsed:5.034902, Percent Complete:0.000000
Time Elapsed:1.62751

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in true_divide
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:32: RuntimeWarning: divide by zero encountered in log10


Time Elapsed:13.377454, Percent Complete:0.000000


Exception in thread Thread-47:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.6/multiprocessing/pool.py", line 405, in _handle_workers
    pool._maintain_pool()
  File "/opt/conda/lib/python3.6/multiprocessing/pool.py", line 246, in _maintain_pool
    self._repopulate_pool()
  File "/opt/conda/lib/python3.6/multiprocessing/pool.py", line 239, in _repopulate_pool
    w.start()
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 105, in start
    self._popen = self._Popen(self)
  File "/opt/conda/lib/python3.6/multiprocessing/context.py", line 277, in _Popen
    return Popen(process_obj)
  File "/opt/conda/lib/python3.6/multiprocessing/popen_fork.py", line 20, in __init__
    self._launch(process_obj)
  File "/opt/conda/lib/python3.6/multiprocessing/pope

Time Elapsed:0.064805, Percent Complete:0.000000


...

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in true_divide
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:32: RuntimeWarning: divide by zero encountered in log10


Time Elapsed:0.076751, Percent Complete:0.000000


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in true_divide
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:32: RuntimeWarning: divide by zero encountered in log10


Time Elapsed:0.000451, Percent Complete:0.000000
Time Elapsed:0.486524, Percent Complete:0.000000


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in true_divide
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:32: RuntimeWarning: divide by zero encountered in log10
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in true_divide
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in true_divide
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:32: RuntimeWarning: divide by zero encountered in log10
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:32: RuntimeWarning: divide by zero encountered in log10
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in true_divide
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:32: RuntimeWarning: divide by zero encountered in log10
/opt/conda/lib/python3.6/site-packages/ipyke

In [ ]:
# pickle file
(pd.concat(results)).to_pickle('../../nowdata/backups/charters_full_2015_250_v2a_unlappedtext_counts3.pkl')